In [ ]:
import sys
sys.path.append('./tool')

from aggregation import *
import analyze
import display

pd.set_option('display.max_rows', 20) 

In [ ]:
exp_root = "/home/ict/appProfile/data/addtion_exp/standard_stress_no_20231023023901"
exp_data = read_from_dir(exp_root)
exp_data.set_workload_preprocess_funcs([
    filter_column_startswith(col_prefix=("stress", "vm", "app")),
    filter_column_useless(excol_prefix=("stress")),
    filter_row_noise(col_prefix=("app")),
])

keys = list(exp_data.exp["info_per_epoch"][0]["workloads"].keys())
keys

In [ ]:
df_workload = exp_data.agg_epoch()
df_workload

# 一、相关性分析
## (1) 皮尔逊相关性

In [ ]:
corr_matrix = analyze.pearson_correlation(df_workload)
corr_matrix

In [ ]:
display.plt_corr_heatmap(corr_matrix)

### 1. 相关性排序

In [ ]:
# 提取相关性大于 c 的指标对
c = 0.95
fcorr_df = analyze.flatten_corr(corr_matrix, similarity_filter = [
    lambda x: x[x.abs() > c].dropna(axis=0, how='all'),
])
fcorr_df

### 2. latency 相关指标

In [ ]:
c = 0.50
latency_corr = analyze.single_corr(corr_matrix, "app_redis_qos_p99_latency_set", similarity_filter = [
    lambda x: x[x.abs() > c].dropna(axis=0, how='all'),
])
latency_corr

In [ ]:
display.plt_by_column(df_workload, columns=list(latency_corr.index))

## (2) 余弦相似性

In [ ]:
corr_matrix = analyze.cosine_similarity(df_workload)
corr_matrix

In [ ]:
display.plt_corr_heatmap(corr_matrix)

### 1. 相关性排序

In [ ]:
# 提取相关性大于 c 的指标对
c = 0.99
fcorr_df = analyze.flatten_corr(corr_matrix, similarity_filter = [
    lambda x: x[x > c].dropna(axis=0, how='all'),
])
fcorr_df

### 2. latency 相关指标

In [ ]:
c = 0.80
latency_corr = analyze.single_corr(corr_matrix, "app_redis_qos_p99_latency_set", similarity_filter = [
    lambda x: x[x.abs() > c].dropna(axis=0, how='all'),
])
latency_corr

In [ ]:
display.plt_by_column(df_workload, columns=list(latency_corr.index))